## combine common data

### feature(주요 common data)합치기-- 하루전과 이틀전의 데이터와 비교

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

In [2]:
# 개장일(date)과 이전 개장일(date_p1), 이전이전 개장일(date_p2)을 dict로 구성
base_data_directory = './data/base_data/stock_market_holydays/'
opening_days_kor = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
opening_days_usa = pd.read_pickle(base_data_directory+'opening_days_usa.pkl') # 미국 개장일 데이터 

In [3]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
              'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
              'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
              'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
common_pkl_usa = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
                  "ixic_future.pkl", "nas.pkl", "snp_future.pkl", "sox.pkl", "spx.pkl", 
                  "us_10yr_bond.pkl", "us_2yr_bond.pkl", "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
                  'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
                  'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
                  'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
common_pkl_kor = [ "kor_10yr_bond.pkl", "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", 
             ]
# common_pkl = [ "dji.pkl" ]

In [4]:
# dict로 만들어 pkl내에 있는 df의 column 이름과 매칭시키기 위하여 구성
common_col_name = {'dji.pkl':'dji', 'dji_future.pkl':'dji_f', 'dxy_future.pkl':'dxy', 
                   'ixic_future.pkl':'ixic_f', 'kor_10yr_bond.pkl':'bond_kor_10',
                   'kor_2yr_bond.pkl':'bond_kor_2', 'kosdaq.pkl':'kosdaq', 'kospi.pkl':'kospi',
                   'krw_rate.pkl':'krw', 'nas.pkl':'ixic', 'snp_future.pkl':'spx_f',
                   'sox.pkl':'sox', 'spx.pkl':'spx', 'us_10yr_bond.pkl':'bond_usa_10',
                   'us_2yr_bond.pkl':'bond_usa_2', 'us_3mon_bond.pkl':'bond_usa_3m',
                   'vix.pkl':'vix', 'wti_future.pkl':'wti',
                   'spsy.pkl':'spsy', 'spny.pkl':'spny', 'spxhc.pkl':'spxhc', 'splrcd.pkl':'splrcd', 
                   'splrci.pkl':'splrci', 'splrcu.pkl':'splrcu', 'splrcs.pkl':'splrcs', 
                   'splrct.pkl':'splrct', 'splrcl.pkl':'splrcl', 'splrcm.pkl':'splrcm', 
                   'ixbk.pkl':'ixbk', 'ixfn.pkl':'ixfn', 'ixid.pkl':'ixid', 'ixis.pkl':'ixis', 
                   'ixk.pkl':'ixk', 'ixtr.pkl':'ixtr', 'ixut.pkl':'ixut', 'nbi.pkl':'nbi', 'bkx.pkl':'bkx'
                   }

In [5]:
directory_for_predict = './data/data_for_ml/predict/'
pkl_common_directory = './data/common_pkl/'
pkl_directory = './data/common_pkl/'

In [6]:
# 주요 지표를 한개의 df로 구성 (휴장일과 개장일이 틀리기 때문에 한국, 미국 각각 구성해야 함)
# _common : 공통으로 사용되는 지표를 의미
def get_common_base(opening_days_common, pkl_list):
    df_base = pd.DataFrame(pd.Series(opening_days_common, name='date')).set_index('date')
    date_compare = pd.DataFrame(pd.Series(opening_days_common, name='date'))
    date_compare['date_p'] = date_compare['date'].shift(1)
    
    date_compare.columns = ['date_c', 'date_p'] #거래일과 바로 이전 거래일
    c_p_dict = date_compare.set_index('date_c').to_dict()['date_p'] # date_c로 date_p 찾기
    p_c_dict = date_compare.set_index('date_p').to_dict()['date_c'] # date_p로 date_c 찾기
    
    for index_name in pkl_list:
        key_name = index_name[:-4]
        col_name = common_col_name[index_name]
        df_temp = pd.read_pickle(pkl_common_directory + index_name)
        df_temp['date'] = df_temp['date'].dt.date # date type으로 변환
        df_temp['temp'] = df_temp[col_name].shift(2)  # 거래일과 이전(-1) 이전(-1) 거래일 데이터와 비교하기 위하여 -2 shift
        df_temp[f'{col_name}_cr_2'] = (df_temp[col_name] - df_temp['temp'])/df_temp['temp']*100 # 이전 거래일 데이터는 "_cr"
        df_new = df_temp[df_temp['date'].isin(date_compare['date_p'])] # 이전 거래일과 비교날짜가 있는 row만 선택
        # 예: 2023-04-10('date_c'), 2023-04-07('date_p') 일 경우, 2023년 4월 7일 데이터는 2023년 4월 10일 row에 작성해야 함.
        # 위 예와 같이 하기 위해 'date_p'에 있는 date로만 df재구성(df_new)

        add_c= df_new['date'].apply(lambda x : p_c_dict[x])  # 하루전 날짜와 매칭한 column 생성 (add_c)
        # 예: 2023년 4월 7일 데이터는 2023년 4월 10일 row에 기입
        df_new.insert(len(df_new.columns)-1, 'date_c', value=add_c)  #SettingWithCopyWarning: error 회피
        df_new.set_index('date_c', inplace=True)

        df_base = df_base.merge(df_new.iloc[:, [-3, -1]], \
                                how='left', left_index=True, right_index=True)
    return df_base

In [7]:
# ***미국 주식 분석용 시작 일자, 마지막 일자  지정 ***********
# ** 한국 공통(kospi 등) 자료는 한국 개장일에 맞춰 분석해야 함.
pkl_name= 'dji.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

dji = df_temp['dji'].iloc[-1]
opening_days_common = list(df_temp['date'].dt.date)

# 오늘 날짜(분석날짜) 추가 필요(미국, 한국 포함)

if dji == None :
    opening_days_common = list(df_temp['date'][:-1].dt.date)
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(opening_days_usa).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(opening_days_usa)[lp1_index])

In [8]:
# 미국 common data df
df_usa = get_common_base(opening_days_common, common_pkl_usa)

In [9]:
# *****한국 주식 시작 일자, 마지막 일자  지정 ***********
pkl_name= 'kospi.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

kospi = df_temp['kospi'].iloc[-1]
opening_days_common = list(df_temp['date'].dt.date)

if kospi == None :
    opening_days_common = list(df_temp['date'][:-1].dt.date)
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(opening_days_kor).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(opening_days_kor)[lp1_index])

In [10]:
# 한국 common data df
df_kor = get_common_base(opening_days_common, common_pkl_kor)

In [11]:
df_all = df_usa.merge(df_kor, how='left', left_index=True, right_index=True)

In [12]:
df_all

,dji_cr,dji_cr_2,dji_f_cr,dji_f_cr_2,dxy_cr,dxy_cr_2,ixic_f_cr,ixic_f_cr_2,ixic_cr,ixic_cr_2,...,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2
date,,,,,,,,,,,,,,,,,,,,,
2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-28,nan%,NaN,nan%,NaN,nan%,NaN,nan%,NaN,nan%,NaN,...,0.45%,0.454133,0.18%,0.179856,nan%,NaN,nan%,NaN,-0.03%,0.005061
2021-12-29,0.26%,NaN,0.29%,NaN,0.12%,NaN,-0.43%,NaN,-0.56%,NaN,...,-0.72%,-0.272480,0.66%,0.839329,1.59%,NaN,0.69%,NaN,+0.21%,0.177067
2021-12-30,0.25%,0.513052,0.27%,0.561114,-0.29%,-0.166513,0.02%,-0.419686,-0.10%,-0.661825,...,-0.73%,-1.446655,-0.36%,0.299222,0.06%,1.650259,-0.89%,-0.208698,-0.29%,-0.076754
2021-12-31,-0.25%,-0.000365,-0.25%,0.024806,0.04%,-0.248439,-0.37%,-0.350255,-0.16%,-0.254473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-18,0.30%,-0.124925,0.27%,-0.178415,0.54%,1.088382,0.04%,-0.176006,0.28%,-0.070275,...,+1.76%,1.762918,+2.12%,2.116564,0.63%,1.705340,0.17%,0.556280,+1.16%,1.158742
2023-04-19,-0.03%,0.266065,-0.01%,0.255604,-0.34%,0.200499,0.05%,0.089142,-0.04%,0.246963,...,+0.12%,1.884498,+0.42%,2.546012,-0.05%,0.573109,-0.19%,-0.015551,0.00%,1.130805
2023-04-20,-0.23%,-0.265300,-0.27%,-0.281286,0.22%,-0.125740,-0.08%,-0.026542,0.03%,-0.004024,...,+0.54%,0.657109,+1.02%,1.441874,0.02%,-0.032984,0.16%,-0.032215,+0.80%,0.803572


In [13]:
# write common analysis data
globals()['df_all'].to_pickle(directory_for_predict + '0_df_common.pkl')
globals()['df_all'].to_csv(directory_for_predict + '0_df_common.csv')